# Preprocessing the data after manual inspection

In [65]:
import warnings
import os, re 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import random
from difflib import SequenceMatcher
warnings.filterwarnings('ignore') # avoid warnings showing up

In [9]:
frames = []
for i in tqdm(range(31)): 
    with open('scraped/data_'+str(i)+'.json') as f:
        dataframe = pd.read_json(f)
        
    frames.append(dataframe)

In [11]:
data = pd.concat(frames, sort=False)


In [15]:
type(data)

pandas.core.frame.DataFrame

In [27]:
politiken = data.loc[(data["Domain"] == "politiken.dk")]

In [52]:
politiken

,level_0,index,Id,Domain,Body,Header,PublicationDate,Uri,Byline,TextHash
0,0,0,842325,politiken.dk,FOR ABONNENTER »Der var overhovedet ingen bes...,Utilfreds passager: »Prøv selv en tur klokken ...,2002-09-17T00:00:00,https://politiken.dk/forbrugogliv/art6088327/U...,Annemette Grundtvig,-346867430
1,1,1,842324,politiken.dk,Klimamonitor Byrummonitor Skolemonitor Sundhed...,Minimalist: »Ofte står jeg og mangler en ting ...,2013-01-18T00:00:00,https://politiken.dk/forbrugogliv/art6287464/%...,Annemette Grundtvig,-1346564151
2,2,2,842323,politiken.dk,Køb abonnement Den familieorienterede type: S...,Digitalt forældreskab: Hvilken type er du som ...,2006-05-17T00:00:00,https://politiken.dk/forbrugogliv/art5934818/H...,Annemette Grundtvig,1473032676
3,3,3,842322,politiken.dk,FAKTA: Kemi i mademballage af papir Forbrugerr...,Ny test afslører: Denne kaffekop og dette suge...,2004-07-19T00:00:00,https://politiken.dk/forbrugogliv/art7278657/D...,Annemette Grundtvig,1097982200
4,4,4,842321,politiken.dk,FOR ABONNENTER Guide til julen Heller ikke i ...,"Pakkeshop, pick-up eller ekspres? Pakkernes sv...",2010-12-19T00:00:00,https://politiken.dk/forbrugogliv/art7524916/P...,Annemette Grundtvig,-2079936539
...,...,...,...,...,...,...,...,...,...,...
305407,307927,9948,742377,politiken.dk,{AVIS} Danmark Da rekordstore konverteringsbøl...,Ny redegørelse: Konverteringsboom gav et løft ...,2013-03-06T00:00:00,https://politiken.dk/indland/art5701476/HK-Str...,Silla Bakalus,2003729823
305410,307930,9953,742372,politiken.dk,Banksektoren Jyske Bank har sat sig tungt på f...,Positive toner fra Jyske Bank sender aktien ti...,2002-08-06T00:00:00,https://politiken.dk/indland/art4850125/Dyrl%C...,Silla Bakalus,550856262
305413,307933,9957,742368,politiken.dk,Væksten i Danmark kan falde mellem 3 og 10 pct...,Nationalbanken: Danmarks økonomi bliver hårdt ...,2010-04-06T00:00:00,https://politiken.dk/kultur/art4856648/Casabla...,Silla Bakalus,-1470063552
305416,307936,9961,742364,politiken.dk,Bo Rygaard FCK-ejeren Parken Sport & Entertain...,Bo Rygaard stopper: Parken Sport & Entertainme...,2024-02-06T00:00:00,https://politiken.dk/viden/art5700989/Stort-pa...,Silla Bakalus,77085385


In [29]:
politiken_bodies = list(politiken["Body"])

In [55]:
subset = politiken_bodies[:50]

In [53]:
bad_strings = ["Køb abonnement", "FOR ABONNENTER", 
               "{AVIS} i 30 dage for kun 1 kr. Få adgang til {AVIS}s digitale univers, og læs artikler, lyt til podcasts og løs krydsord"]

In [54]:
bad_strings

['Køb abonnement',
 'FOR ABONNENTER',
 '{AVIS} i 30 dage for kun 1 kr. Få adgang til {AVIS}s digitale univers, og læs artikler, lyt til podcasts og løs krydsord']

In [75]:
len(subset[1])

721

In [90]:
#subset[10, 12, 20] er et eksempel på artikel der er tydeligt afskåret af en paywall

subset[12]

' Køb abonnement FOR ABONNENTER Både børn med en egentlig diagnose og ’almindelige’ børn, der trænger til mere ro og koncentration, kan have glæde af justeringer i kosten, lyder hovedbudskabet fra ernæringsterapeut Karen Nørby. Som afslutning på sit studium skrev hun opgave om kost og hyperaktivitet og fandt ud af, at der er forsket en del i kostens betydning for børn og unge med svært styrbar energi og adfærd. »Alene de grundlæggende biokemiske forhold omkring hjernens funktion giver god grund til at se på ernæring. Men i Danmark har i vi ikke tradition for at inddrage kost i eventuelle behandlingstilbud«, siger Karen Nørby. Hun har skrevet en bog, som forklarer, hvorfor kostændringer kan være velbegrundede. Bogen dokumenterer med eksempler, at de for nogle er meget virkningsfulde. Kostplaner, opskrifter og forslag til kosttilskud viser vej, hvis forældre har lyst til at prøve, om de på dette felt selv kan skabe bedre trivsel for deres børn. Følsom eller robust »Nogle gange skal der i

In [82]:
subset[2]

" Køb abonnement Den familieorienterede type: Slægt skal følge slægters gang For denne type er det naturligt at lægge et billede af barnet ud hurtigt efter fødslen. Nogle offentliggør scanningsfoto og viser med tekster som »vores lille nye familiemedlem«, at de allerede inden fødslen tænker på sig selv som en familie. Der postes løbende billeder af barnet. Af hverdagsøjeblikke, der signalerer intimitet, samvær og opbygning af relationer. Opslag er ofte knyttet til noget rituelt/ceremonielt og milepæle: Hjemme fra fødegangen. Første tur i barnevognen. En måned gammel. Debut til babysvømning. Barnedåb. Halvårsfødselsdag. Fejringer. Højtider. Formålet er at etablere en familiefortælling på Facebook og skabe en familiær identitet ved at dele og vise sit nye familieliv frem. Man markerer et tilhørsforhold til en bestemt slægt. Man går op i traditioner og har et hverdagsliv, der er centreret omkring familien. Der er både kvinder og mænd i denne gruppe, hvor det er almindeligt, at begge foræl

In [71]:
#This function will identify phrases that are repeated throughout the domains. 
#However, it will have multiple duplicate matches, making the conversion slow... Oh well...
def get_junk_by_domain(domain, verbose=False):
    bodies = data[data.Domain==domain].Body.to_list()
    junk = {}
    if verbose:
        print("Started on", domain)
    for i in tqdm(range(len(bodies)//20)):
        #Loop through a quarter of the bodies from the domain.
        randomindex = None

        while True:
            #Find a random index for a article
            randomindex = random.randint(0,len(bodies)-1)
            if randomindex != i:
                break
        
        #We now have to articles to compare
        string1 = bodies[i]
        string2 = bodies[randomindex]

        #match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))
        
        #Find the indices for the three longest identical substrings in the two articles.
        matches = SequenceMatcher(None, string1, string2).get_matching_blocks()
        
        #Last match is always useless. Loop through the matches.
        for match in matches[:-1]:
            if match.size > 20:
                #If match is longer than 20 characters, extract string from indices and add to dict.
                matchstr = string1[match.a: match.a + match.size] 
                if matchstr not in junk:
                    junk[matchstr] = 1
                else:
                    junk[matchstr] += 1
    
    if verbose:
        print(f"Found {len(junk)} sentences to remove in {domain}...")
    
    keys = list(junk.keys())
    if verbose:
        print(keys)
    junk_list = []
    
    #Loop through the junk found and add to a list, if they occur more than 2 times.
    for key in keys:
        if junk[key] > 2:
            junk_list.append(key)
    
    if verbose:
        print(f"Narrowed that down to {len(junk_list)} junk sentences.")
    
    return junk_list

In [72]:
# domain_junk = {}
# for domain in tqdm(data.Domain.unique()):
#     domain_junk[domain] = get_junk_by_domain(domain)

In [73]:
get_junk_by_domain("politiken.dk")

[' Køb abonnement FOR ABONNENTER ',
 ' {AVIS}s digitale univers',
 'Prøv {AVIS} i 30 dage for kun 1 kr. Få adgang til ',
 '{AVIS}s digitale univers, og læs artikler, lyt til podcasts og løs krydsord.',
 '{AVIS}.dk i 3 måneder ',
 '{AVIS}.dk i 3 måneder - kun 299 kr. Læs hele artiklen nu',
 '{AVIS}- Køb abonnement FOR ABONNENTER ',
 '{AVIS}- Køb abonnement ']